<a href="https://colab.research.google.com/github/Karlajack/Python_classes/blob/main/Advance_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D22c389643dbb81c235958e4740f1cdc3491ca833b0f0afdaaf928ad38e714251fb07b5ebce902e0412c21de9e4c12446e3d77d9653ec32c61233b69d934dae0271edd93bcdc6d9b3af612b0e5042747557ce2f7d1efa9d54ea7a412507d5ecfc57689e5399a35700bfa665e2d4562de51c28923b5a1fbc78ab3bf3c811196e999b3f8d37abbdb251c90198e2030ce8736f777b554c47d0d607fb2831d1cbd856b4ea8ff49867db4470186f01f6fa6c82ce59cdf7c3da0c318194027cd8e3ad6298077a2fd9ffe03fcabd09a9a6bd48a30da370618862b61cde9a5a0b8c5e5f2412ddb4a78b7dd6d2b8bc3a2083ff80de889f979f20a40b0fdfd5e93a9665b5d9,california-housing-prices:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5227%2F7876%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5be90a802a01b380069d5ce6ba5dce4c3f0546e6cc7b15ea5644fe68daadb73c0ffc8f673112ea9e06bf70ad951c46552a07638b1a087c21aee6c87a0036dfced99dd483c62629da513941b293a48a916441b751cd57678c4b1de7ee986bee094aba2e068ba7efec8d4fc91edde8bbf210aeb3a16b3c07c4bce614f4e1b4dd99623b17a65ac62746c5d867a50789749e4659b96e217b82a27e0fe02c1bac7dbeb1f3e7e1fcdbf41b24f0cac6468270b0c1f39f74426366dd674616df2f595526ce0e8ae684e0b0fcd8a3225cc39359f9b9f4b6a1475f95914071d63316a9e2818590673c99e3d27876dd2ffbc63e60489887e0e7f61f5dcd00d1eda38d80a555,creditcardfraud:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F310%2F23498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D97c6fd95ca5587eca2f9773356885cfc05441cfdea1997d22c12d864b939763626b6c289aacd769522d7a517df19a743b5b6c9e78d60fc2c42a5f8331bee6b8ad3b0f45ddd6b2a642d41ac6f47d9c2fe05da5678ba68820c9c38488e6c4eab286f6325b8809e50a4969023d19bab5178aa4f34cd79bd814cc3dccc1322ed18ed93ae2d0910fbcd7d7470f9a5a1cafd29993bcc9dba72347388e918323ac82dcdd4d296962fd1e7f460fe7fd4de9463d6608f13e1ca21779c89f58de6557fe093feb3312f13cfbab7e554f89f1a95990306f758e05dbd2a4c8bb5e65b9cce54caa3051ae71e5c1b983e97d48c6678e45f7ba32bbf614f7b2b3bd18a8590a44f16,melbourne-housing-snapshot:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2709%2F38454%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4a239d34b0455130b8dd3b90561fb71696a1086ffcbe283fb4b37589de1c81971c4f7b969d0d0328d88b385b68439e8b38129bd1383cec6cd75b3edac6c2d7f8c5cf73a60d06859d7fd3eba5fae4d1cb7495846aa89215bccdc9736c28f2d04e2a9aab5bf2fa43b86114c4c8b3828dad555d6522a8a872d26edcb9d1a369e2e2af533eb8340cf1126024aa6a275b119eb59837a0f35107e36a045fc0dad82bbf4a01b527ccaf9d093102da93b2936656ce492d12818bb3729220a11fa3749dbbc13438c4bc782ef1a3961a5b6afd806c1255d363ffc1e40284d5902502008bc4376bc71ec8f2158f9d71e432519f3c38049d622769ff631ffe68141d94afc627,loan-defaulter:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F807638%2F1383922%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D186ba733b5b0b31256e9a0305c229a17e4dee7a2fe75315bed8b738c764461f337ee38a68409fb04499800a0b473527f4ec33f9dffb1a4b0b4832356445e1524116ceffe5506512c8034267fa8940db49be4adaf056cded581a45adc1f7f5b4ac78f693abfb0b25a0358789d56c3fe0814a65b5163cb8d8e01512d7d7e9ad3486b282dbf5d427f56fdbfb3d71606b3e859ae0634f6465bcc50d2e152232c80964aab50ef87e4581d73dce8fb3b059d2317a13de2aca5993d1c7440ffeb2e6b7b6fbf00bfe54b34a3a5e0651dc1ea1ade7016dab6b47a749bd76895440235c80434d123e41d2205f3425854d96509f9a365a5671a58fe28c177f24013c63b4f62,credit-card-customers:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F982921%2F1660340%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D80d0d15bdf209708b899207f184b56e7793064101082d174f7079185461da7a21cf1ea9d0801fdd3c897a01a7637b0b2c85db949ea517eb0033ee6187ebd8fd80f2a1f763a978eb9fc0bd1b45339281fb24dd5512264d011657650e4381a64ed9becd1cd424bffd6bec3810abbe247fe742ef0c77fdd39b5200cc7189f755b719b06998e80cd8e5ca49400870daf8a9806d758daa8ee80da8c10a8bb37c8e5577789427f997c80d6d9d97773502a29fbd9904c8e5c2c51c3ad4ea4ca8b7b819c03a22d479752ef2ed4792fdc83b50f05ba5d2eb75ad49e13bf76a989854ed5f712a77604abe35a1df6b375398275c1eba4cfae9dfff1c4ce595e34bcbaad7c93,hr-analytics-job-change-of-data-scientists:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1019790%2F1719283%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8a67ca949183749b2aebf8a453570de1f973bad54263e428fb5df1efce6e0303f0af58c2f16dda073e47d2aca85fc7eb24bdd27fd3ab828eabc321c8902b80a3f272150bd5bd0a781a01f0a9161059cb6ae5cc2ec4c6fa276700b536ba27155bd3315db01232221966bae4abdd0b168a07e14c454f5bad497687faaa760754bed6e35dd7b2ae9ce5b272c1fd593e8a86421a3220c801823e17c64a2c8f305d7183ecaf7ceb2089b933b3b72906b011038c3668954b540891fbf370dec2160ce1288db9c40f8dbe12b794b108ef96c56fe83783ae9d3084ea3fce9751b0f79bcd36e85ee07c91d8833aea7674bbe2acb944baa3831e4c0aff3723d6dcc2a99126,loan-default-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1897041%2F3106815%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9d46865213a94d274fbae84dcf0e08be44c09341325a4cb08009c383dce5a336f7f2d1603f66d56e9351ae0e3f6a7dd349021f53768e1dd1ffd16f040ede4d556ce7c60752fd643bc785a079eef89607ff87c89a29e56f54e19236739a7900be63f4c06bf90a684a20d22783b6ea3acadd2d73d71096fef4360059f8d92a0b413841399d01d8136e6c6978dc6e4ca1afe1c15821c5ebb2cba2d67651644d1707c80edb1f393b23cf53145e2c8e26631fcda22a8e3f86705bb73448da7db576728dd44bb5eb154b809cb9242137c1224f23e05b5dec90be32256389085449b06461eb6abaf22d5068a9f4f144ed7cf9fe8f814ba592df210aaa45888bb999c477,personal-key-indicators-of-heart-disease:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1936563%2F3191579%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7d0edac928ce610c01e0a60a69ea8921edb564d3c1f00f0d409ffae981dac4b90bca95c9f22554ce8845f7c3aadbe4c836c6055f0018c944ac0e61d8c9851d8d2046d8ecca579d82fd56bb62a55ef847e7c75f5bfa70282bfccca38c258ac53d7efc1735238f9ce181012578072c52bc14198f2679cb1fec0c38ac214aedb4da0a75d7f2ef803217944d9e21b43c32bbccf1e7bfad42c52fe7d93acce5bea18a1dd4e01ffabd41dd3b1f2a323f0ed5ad833fdae6e3ecff5d1889cd625dd19ed5f2d5d7413007a7b00281e53d303cb21caabb6c247b14c49a44cedde81f49558af7d306d4187dd9d39ad49ea601dae7c92c463237950a20f894eb443aedabe023,latest-laptop-price-list:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2037622%2F3402351%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5e7b1ea1aefd500cb8f36947089ff328301b489194a1cee2790f58b83a0e64b38ed385f3cb9a3646d48c5aef3a2e32ba759b66e06a26aaa1074e4e55832db0b43cb5467945ab852c30849bb6a9a4f7f8d6c92bad86d546d83cd942944dc8d16ff5963313f2f24e7c70794e077e51d203a186c0a325347c65f34c3182f143a4a0350168d8c7409500f8f79ec20b598a00e335529a202221c913929a69337768bf48c0ba0f5a096fb9504b87d274d80e1d09c2a7f7fe06cdc90e7d9c4e00bed0888b68f58edb71e8ed2a670bde1a8ae93dcf19813b1c7c916c1b5ffb9506bc04808076d1027a1bc52726ea5d9ad1b80e8e6145d02871de1083cb86a19f0909f727,cost-of-living-index-2022:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2027131%2F3430638%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2bab40057351db05bfef262fd2d44e43652db176e9377e8f9cf7ff5f3f0b7ccae0c2c6bdf7266bdb414d2a2c66755e23be5e4585ae00b34dce834dfc163cf0d176f3afcf79b0521f10cfd0364511a68f879bd8a1f4c3ae175b21196beb584dee5230d7326d8653b1c8f22d2acefdb78f31be1feb89cd033ac6bee3e6675e3643e4d08f8c0c179e56c57cb9fa6e10a6af66f8deedf780541d250cec936dcc3a34d2ab17f1395b733b1cd015ecab0bb2823bb60fb47bb8bd2cded71531f3d4f126d5ae41934c7bfeab87d1310db8f60e7bb716ef6066919bec842d546c13e0b81742c33f49909903f38458fdc743ad88e79bec75ac9d0e0935889a22cdbff49af6,historical-commodity-prices-from-20002022:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2040950%2F3432853%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T103118Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D081850404897a3647b5d86668239ead4cd3164154869b6d03037a2e32aecd5188c0a13521249d6d5258424a4657d3a6be085a29f5aa780bc35e3284a8a98abcfe60389bf41c6cb8e6017c59623bb0d8a17f65924477e2898f8d00fc92caf0411f7f15b569917a46e67e0556298971ff277e7c2e5f290a10d0c71ac377cc806633912f5695ea2b7fd9d765bdf15118d59724771a6916d1941750aaed382d2b28c2833fb21a878d50357e54ca84f327719c470785c38eaaa65b003977cb001e0acbd30474d3c3b186f148ea009404860f7959ffc4d3df250526219ac00c3a3f89298255d6ced61470c7fc3334b39a08e03f312b45920c6e45bcca46c3bcd83fb1e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


 <h1 style="text-align: center;" class="list-group-item list-group-item-action active">Table of Contents</h1>

<ul style="list-style-type:none;">
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab"
                aria-controls="settings">1. Introduction<span class="badge badge-primary badge-pill">1</span></a>
        </li>
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab"
                aria-controls="settings">2. Handling Missing Values<span
                    class="badge badge-primary badge-pill">2</span></a>
            <ul style="list-style-type:none;">
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1" role="tab"
                        aria-controls="settings">2.1 Visualizing Missing Data <span
                            class="badge badge-primary badge-pill">3</span></a>
                    <ul style="list-style-type:none;">
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.1"
                                role="tab" aria-controls="settings">2.1.1 Matrix <span
                                    class="badge badge-primary badge-pill">4</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.2"
                                role="tab" aria-controls="settings">2.1.2 Correlation Heatmap <span
                                    class="badge badge-primary badge-pill">5</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.3"
                                role="tab" aria-controls="settings">2.1.3 Dendrogram <span
                                    class="badge badge-primary badge-pill">6</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.4"
                                role="tab" aria-controls="settings">2.1.4 Simple Numeric Summaries<span
                                    class="badge badge-primary badge-pill">7</span></a></li>
                    </ul>
                </li>
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2" role="tab"
                        aria-controls="settings">2.2 Method to Handle Missing Data<span
                            class="badge badge-primary badge-pill">8</span></a>
                    <ul style="list-style-type:none;">
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2.1"
                                role="tab" aria-controls="settings">2.2.1 Deletion of Data <span
                                    class="badge badge-primary badge-pill">9</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2.2"
                                role="tab" aria-controls="settings">2.2.2 Encoding Missingness<span
                                    class="badge badge-primary badge-pill">10</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2.3"
                                role="tab" aria-controls="settings">2.2.3 Imputation Methods<span
                                    class="badge badge-primary badge-pill">11</span></a></li>
                    </ul>
                </li>
            </ul>
        </li>
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3" role="tab"
                aria-controls="settings">3. Encoding Categorical Attributes<span
                    class="badge badge-primary badge-pill">12</span></a>
            <ul style="list-style-type:none;">
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1" role="tab"
                        aria-controls="settings">3.1 Supervised Encoding Methods <span
                            class="badge badge-primary badge-pill">13</span></a>
                    <ul style="list-style-type:none;">
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1.1"
                                role="tab" aria-controls="settings">3.1.1 Likelihood Encoding <span
                                    class="badge badge-primary badge-pill">14</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1.2"
                                role="tab" aria-controls="settings">3.1.2 Target Encoding <span
                                    class="badge badge-primary badge-pill">15</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1.3"
                                role="tab" aria-controls="settings">3.1.3 Deep Learning Methods <span
                                    class="badge badge-primary badge-pill">16</span></a></li>
                    </ul>
                </li>
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.2" role="tab"
                        aria-controls="settings">3.2 Approaches for Novel Categories<span
                            class="badge badge-primary badge-pill">17</span></a>
            </ul>
        </li>
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#4" role="tab"
            aria-controls="settings">4. References <span
                class="badge badge-primary badge-pill">18</span></a> </li>


 </ul>



<h1  style="text-align: center" class="list-group-item list-group-item-action active">1. Introduction</h1><a id = "1" ></a>



We all know the Importance of good features for machine learning models. In Machine learning task we have features which we need to process to make them good and this is done by data preprocessing  tasks.

**Data Preprocessing** : Data preprocessing is a process of preparing the raw data and making it suitable for a machine learning model. It is the first and crucial step while creating a machine learning model. When creating a machine learning project, it is not always a case that we come across the clean and formatted data. And while doing any operation with data, it is mandatory to clean it and put in a formatted way. So for this, we use data preprocessing task.

A real-world data generally contains noises, missing values, and maybe in an unusable format which cannot be directly used for machine learning models. Data preprocessing is required tasks for cleaning the data and making it suitable for a machine learning model which also increases the accuracy and efficiency of a machine learning model.

Data Preprocessing involves below steps:

- Getting the dataset
- Importing libraries
- Importing datasets
- Finding Missing Data
- Encoding Categorical Data

Many of us know traditional approaches for above listed steps but in this notebook I will discuss some different approaches which could be game changer for your next project.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/loan-default-dataset/Loan_Default.csv")
df.head()

In [ ]:
df.info()

<h1  style="text-align: center" class="list-group-item list-group-item-action active">2. Handling Missing Values</h1><a id = "2" ></a>

Missing data are not rare in real data sets. In fact, the chance that at least one data point is missing increases as the data set size increases. Missing data can occur any number of ways, some of which include the following.

- Merging of source data sets
- Random events
- Failures of measurement



<h2  style="text-align: center" class="list-group-item list-group-item-success"> 2.1 Visualizing Missing Data</h2><a id = "2.1" ></a>


Visualizations as well as numeric summaries are the first step in understanding the challenge of missing information in a data set. For small to moderate data (100s of samples and 100s of attributes), several techniques are available that allow the visualization of all of the samples and Attributes simultaneously.

In this notebook I'll Cover Following visualizations for missing values:-
- Matrix
- Correlation Heatmap
- Dendrogram
- Simple numerical summaries

Question may arise that why we need Visualizations?
Because it is wise to explore relationships within the attributes that might be related to missingness.

<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 2.1.1 Matrix</h3><a id = "2.1.1" ></a>

It is the nullity matrix that allows us to see the distribution of data across all columns in the whole dataset. It also shows a sparkline (or, in some cases, a striped line) that emphasizes rows in a dataset with the highest and lowest nullity.

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(df)
plt.figure(figsize = (15,9))
plt.show()

From the above plot we can interpret our dataset has lots of missing values in it

<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 2.1.2 Correlation Heatmap </h3><a id = "2.1.2" ></a>

Correlation heatmap measures nullity correlation between columns of the dataset. It shows how strongly the presence or absence of one feature affects the other.

Nullity correlation ranges from(-1 to 1):
- -1 means if one column(attribute) is present, the other is almost certainly absent.
- 0 means there is no dependence between the columns(attributes).
- 1 means if one column(attributes) is present, the other is also certainly present.

Unlike in a familiar correlation heatmap, if you see here, many columns are missing. Those columns which are always full or always empty have no meaningful correlation and are removed from the visualization.

The heatmap is helpful for identifying data completeness correlations between attribute pairs, but it has the limited explanatory ability for broader relationships and no special support for really big datasets.

In [ ]:
msno.heatmap(df, labels = True)

From above visualization we can easily interpret missingness of attribute rate_of_interest and upfront_charges is dependent on each other(correlation value = 1) means if one will be present another will be present.

<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 2.1.3 Dendrogram </h3><a id = "2.1.3" ></a>


The dendrogram shows the hierarchical nullity relationship between columns. The dendrogram uses a hierarchical clustering algorithm against one another by their nullity correlation.

[More about Hierarchical Clustering Algorithm](http://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html)

In [ ]:
# Columns having missing values
missing_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
missing_columns

In [ ]:
msno.dendrogram(df[missing_columns])

We interpret the dendrogram based on a top-down approach, i.e., to focus on the height at which any two columns are joined together with matters of nullity. More will be the height less will be the relation and vice versa is also True.

For example if we see pair of attributes LTV and property value has height 0 implies they are highly correlated in case of nullity. Similarly attribute LTV and rate_of_interest have maximum height implies they are less correlated with each other.  

<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 2.1.4 Simple Numerical Summaries </h3><a id = "2.1.4" ></a>


Moving Forward lets try to analyse numerical summary of missing attributes. Simple numerical summaries are effective at identifying problematic predictors and samples when the data become too large to visually inspect.

In [ ]:
def get_numerical_summary(df):
    total = df.shape[0]
    missing_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    missing_percent = {}
    for col in missing_columns:
        null_count = df[col].isnull().sum()
        per = (null_count/total) * 100
        missing_percent[col] = per
        print("{} : {} ({}%)".format(col, null_count, round(per, 3)))
    return missing_percent

In [ ]:
missing_percent = get_numerical_summary(df)

Now I guess visualization part is done lets move forward to methods which we can use to handle missing values.


<h2  style="text-align: center" class="list-group-item list-group-item-success"> 2.2 Methods to Handle Missing Data</h2><a id = "2.2" ></a>

As we Know if our data has missing values than our model will not train except few models which can tolerate them like some tree based models but the point is we want to handle this and how can we handle them. So, in this notebook to handle missing data I will discuss following techniques :-

- Deletion of Data
- Encoding Missingness
- Imputation Methods


<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 2.2.1 Deletion of Data </h3><a id = "2.2.1" ></a>



The simplest approach for dealing with missing values is to remove entire attribute(s) and/or sample(s) that contain missing values. However, one must carefully consider a number of aspects of the data prior to taking this approach. For example, missing values could be eliminated by removing all predictors that contain at least one missing value. Similarly, missing values could be eliminated by removing all samples with any missing values.

**Note: When it is difficult to obtain samples or when the data contain a small number of samples (i.e., rows), then it is not desirable to remove samples from the data.**

Consider this small intuition shown below

Let M = Number of Samples(rows).\
and Let N = Number of Attributes(columns).


Case 1: Deletion of Attributes

If N has range of [1-10]\
Then don't delete the attribute that contain missing values but if that attribute has missing values around 80-90% then deletion of that attribute will be good option instead of just predicting values of those 80-90% data based on that 10-20% data.

Case 2: Deletion of Samples

If M is a large number according to your task\
Then deletion of sample can be a Good step but if that sample has few missing values with respect to attribute, then you should consider methods to fill those missing values.

Lets move on to the implementation part, I will just show how to delete data for both cases but you can interpret more according to your tasks.

**Deletion of an Attribute**

According to Simple numerical Summaries the attribute Upfront_charges has largest missing values percentage of (26.664%) which is not ideal percentage to remove a feature but just for sake of implementation I will remove that feature.

In [ ]:
df_temp = df.copy()

In [ ]:
# Threshold to remove attribute having missing values greater than threshold
ATTRIBUTE_THRESHOLD = 25 #25% in this case

for col, per in missing_percent.items():
    if per > ATTRIBUTE_THRESHOLD:
        df_temp.drop(col, axis = 1, inplace = True)


By generating numerical summary of df_temp we can see now attribute Upfont_chargers being removed from the dataset as it has missing values percentage greater than threshold we defined

In [ ]:
_ = get_numerical_summary(df_temp)

In [ ]:
del df_temp

**Deletion of the Samples**

We will try to delete those samples having missing values in more than 5 attributes  

In [ ]:
df_temp = df.copy()

In [ ]:
# Getting Missing count of each sample

for idx in range(df_temp.shape[0]):
    df_temp.loc[idx, 'missing_count'] = df_temp.iloc[idx, :].isnull().sum()

In [ ]:
# Threshold to remove samples having missing values greater than threshold
SAMPLE_THRESHOLD = 5

print("Samples Before Removal : {}".format(df_temp.shape[0]))

df_temp.drop(df_temp[df_temp['missing_count'] > SAMPLE_THRESHOLD].index, axis = 0, inplace = True)

print("Samples After Removal : {}".format(df_temp.shape[0]))


In [ ]:
del df_temp

<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 2.2.2 Encoding Missingness </h3><a id = "2.2.2" ></a>



When an attribute is discrete in nature, missingness can be directly encoded into the attribute as if it were a naturally occurring category. For example in this dataset the attribute loan_limit has 3344 missing values so we can assign some new category to these missing values.

In [ ]:
cat_missing_cols = [col for col in missing_columns if df[col].dtype == 'object']
cat_missing_cols

In [ ]:
df.loan_limit.value_counts()

In [ ]:
df[cat_missing_cols] = df[cat_missing_cols].fillna('Missing')
df.loan_limit.value_counts()

In [ ]:
df[cat_missing_cols].info()

<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 2.2.3 Imputation Methods</h3><a id = "2.2.3" ></a>



Another approach to handling missing values is to impute or estimate them. Imputation uses information and relationships among the non-missing
attributes to provide an estimate to fill in the missing value.

In this section we will work on imputation models which will help us impute missing values by extracting interesting patterns from attributes which don't have missing values at that point on time.


- Within a sample data point, other variables may also be missing. For this reason, an imputation method should be tolerant of other missing data.

- Imputation creates a model embedded within another model. There is a prediction equation associated with every attribute in the training set that might have missing data. It is desirable for the imputation method to be fast and have a compact prediction equation.

- Many data sets often contain both numeric and discrete attributes. Rather than generating dummy variables for discrete attributes, a useful imputation method would be able to use attributes of various types as inputs.

- The model for predicting missing values should be numerically stable and not be overly influenced by outlying data points.

Virtually any machine learning model could be used to impute the data. Here, the focus will be on several that are good candidates to consider.

Question arise if an attribute has missing values around 50-60% then can we use imputation methods? And the answer is it depends upon datasets which we are using because an attribute with 60% missing values may has very good correlation with some other attribute which can be helpful to fill those missing values on the other side if let say some column like ID column which is independent of all columns has missing values around 10% using imputation methods we may not get results we wanted.

In this notebook we are gonna work on following imputation methods:-

- KNN for Imputation
- Tree Based Imputation
- Linear Models for Imputation   

<h3>(a) K-Nearest Neighbors(KNN) for Imputation</h3>

When the training set is small or moderate in size, K-nearest neighbors can be a quick and effective method for imputing missing values. This procedure identifies a sample with one or more missing values. Then it identifies the K most similar samples in the training data that are complete (i.e., have no missing values in some columns). Similarity of samples for this method is defined by a distance metric. When all of the predictors are
numeric, standard Euclidean distance is commonly used as the similarity metric.

After computing the distances, the K closest samples to the sample with the missing value are identified and the average value of the predictor of interest is calculated. This value is then used to replace the missing value of the sample.

In [ ]:
from sklearn.impute import KNNImputer

df_temp = df.copy()

As we haven't done categorical encoding yet (We'll cover it in next section) so, for time being lets impute on numerical data only later we will impute on full data after encoding.

In [ ]:
num_cols = [col for col in df_temp.columns if df_temp[col].dtype != 'object']
print(num_cols)
df_temp = df_temp[num_cols]

In [ ]:
# Initializing KNNImputer
knn = KNNImputer(n_neighbors = 3)

knn.fit(df_temp)

In [ ]:
X = knn.transform(df_temp)

In [ ]:
df_temp = pd.DataFrame(X, columns = num_cols)
df.info()

In [ ]:
del df_temp

<h3>(b) Trees</h3>

Tree-based models are a reasonable choice for an imputation technique since a tree can be constructed in the presence of other missing data. While a single tree could be used as an imputation technique, it is known to produce results that have low bias but high variance. And we all know who kills the bias its Ensembles of trees.

Random forests is one such technique and has been studied for this purpose. However, there are a couple of notable drawbacks when using this technique in a predictive modeling setting. First and foremost, the random selection of predictors at each split necessitates a large number of trees (500 to 2000) to achieve a stable, reliable model. This can present a challenge as the number of attributes with missing data increases since a separate model must be built and retained for each predictor. Also Random forest will have heavy computations.

A good alternative that has a smaller computational footprint is a bagged tree. A bagged tree is constructed in a similar fashion to a random forest.
The primary difference is that in a bagged model, all attributes are evaluated at each split in each tree. The performance of a bagged tree using 25–50 trees is generally in the ballpark of the performance of a random forest model. And the smaller number of trees is a clear advantage when the goal is to find reasonable imputed values for missing data.




In [ ]:
missing_columns

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dr = DecisionTreeRegressor()

In [ ]:
income = df['income']

As we have to encode categorical variables into numerical data to use sklearn's tree based models so for the time being I am encoding categorical variables using Label Encoding Method

In [ ]:
df_temp = df.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

cat_cols = [col for col in df.columns if df[col].dtype == 'object']

for col in cat_cols:
    df_temp[col] = lb.fit_transform(df_temp[col])


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor


def tree_imputation(df):
    missing_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
    non_missing_cols = [col for col in df.columns if df[col].isnull().sum() == 0]
    # num_cols = [col for col in missing_cols if df[col].dtype != 'object']

    # df = df[num_cols]
    for col in missing_cols:

        # Defining a new bagging model for each attribute
        model = BaggingRegressor(DecisionTreeRegressor(), n_estimators = 40, max_samples = 1.0, max_features = 1.0, bootstrap = False, n_jobs = -1)

        col_missing = df[df[col].isnull()]
        temp = df.drop(df[df[col].isnull()].index, axis = 0)

        # print(temp.columns)
        # X = temp.drop(col, axis = 1)
        X = temp.loc[:, non_missing_cols]
        y = temp[col]

        model.fit(X, y)

        y_pred = model.predict(col_missing[non_missing_cols])
        # col_missing[col] = y_pred

        df.loc[col_missing.index, col] = y_pred

    return df


In [ ]:
df_new = tree_imputation(df_temp)
df_new.info()

In [ ]:
msno.bar(df_new)
plt.show()

We can see all missing values from the dataset are gone. Now as we temporarily encode categorical variables because we will encode them in later section so lets decode them.

In [ ]:
df_new = pd.concat([df[cat_cols], df_new.drop(cat_cols, axis = 1)], axis = 1)
df_new.head()

In [ ]:
df_new.info()

<h3>(c) Linear Methods</h3>

When a complete Attribute shows a strong linear relationship with a attribute that requires imputation, a straightforward linear model may be the best approach. Linear models can be computed very quickly. Linear regression can be used for a numeric attribute that requires imputation.
Similarly, logistic regression is appropriate for a categorical attribute that requires imputation.

Let say feature rate_of_interest and Interest_rate_spread are dependent features means one feature can be defined using other. If feature rate_of_interest has missing values than it can be imputed using simple linear model trained on Interest_rate_spread.

<h1  style="text-align: center" class="list-group-item list-group-item-action active">3. Encoding Categorical Attributes</h1><a id = "3" ></a>


Categorical Features are those that contain qualitative data.This Section focuses primarily on methods that encode categorical data to numeric values.

Categorical variables/features are any feature type can be classified into three major types:

- Nominal
- Ordinal
- Binary

**Nominal variables** are variables that have two or more categories which do not have any kind of order associated with them. For Example if our dataset has any 4 types of colors, i.e. Red, Blue, Orange, Green it can be considered as a nominal variable.

**Ordinal variables** on the other hand, have “levels” or categories with a particular order associated with them. For example, an ordinal categorical variable can be a feature with three different levels: low, medium and high. Order is important.

**Binary Variables** are same as nominal variables but with only categories For example, if gender is classified into two groups, i.e. male and female.

For Nominal Variables We generally uses Label Encoding Scheme in which we encode each category by just converting it to some integer values this kind of encoding can work in case of Ordinal variables but **for label encoding it has the disadvantage that the numeric values can be misinterpreted by algorithms as having some sort of hierarchy/order in them**. This ordering issue is addressed in another common alternative approach called 'One-Hot Encoding'.

One-Hot-Encoding has the advantage that the result is binary rather than ordinal and that everything sits in an orthogonal vector space. **The disadvantage is that for high cardinality, the feature space can really blow up quickly and you start fighting with the curse of dimensionality.**

Another big issue with encoding schemes is new category or while splitting data in train/validation/test set all samples of the rare classes may split into validation/test set then during it will raise error while predicting.  


Because of few potential issues with traditional approaches we now need to search for some unique approaches some of them are listed below:-

- Supervised Encoding Methods
- Approaching for Novel Categories

Starting from Supervised Encoding Methods lets define it first


<h2  style="text-align: center" class="list-group-item list-group-item-success"> 3.1 Supervised Encoding Methods</h2><a id = "3.1" ></a>


There are several methods of encoding categorical variable to numeric columns using the output data as a guide (so that they are supervised methods). In Supervised Techniques we will discuss following methods to encode categorical variable:-

- Effect or Likelihood Encoding  
- Target Encoding
- Deep Learning Methods



<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 3.1.1 Likelihood Encoding</h3><a id = "3.1.1" ></a>



The effect of the factor level on the output data is measured and this effect is used as the numeric encoding. Here effect of that particular category on output data can be calculated using simple linear models or mean, mode and median methods.

For classification problems, a simple logistic regression model can be used to measure the effect between the categorical outcome and the categorical predictor. After Computing Effects we will compute log-odds of those effects, If the effect is p, the odds of that event are defined as p/(1 − p) and log odds by simply taking log of odds. After when we get the log odds of each category we than encode each category with them using map function.     


In [ ]:
df_temp = df_new.copy()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


## Again we have to temporarily encode variables
lb = LabelEncoder()

cat_cols = [col for col in df_temp.columns if df_temp[col].dtype == 'object']

for col in cat_cols:
    df_temp[col] = lb.fit_transform(df_temp[col])


In [ ]:
def likelihood_encoding(df, cat_cols, target_variable = "Status"):
    # cat_cols.remove(target_variable)
    df_temp = df.copy()
    for col in cat_cols:
        effect = {}
        print(col)
        for category in df[col].unique():
            print(category)

            try:
                temp = df[df[col] == category]
                lr = LogisticRegression()
                X = temp.drop(target_variable, axis = 1, inplace = False)
                y = temp[target_variable]
                # print(temp.drop(target_variable, axis = 1).isnull().sum())
                lr.fit(X, y)

                effect[category] = accuracy_score(y, lr.predict(X))
            except Exception as E:
                print(E)

        for key, value in effect.items():
            effect[key] = np.log(effect[key] / (1 - effect[key] + 1e-6))

        df_temp.loc[:, col] = df_temp.loc[:, col].map(effect)
    return df_temp

In [ ]:
df_temp = likelihood_encoding(df_temp, cat_cols)

In [ ]:
df_temp.head()

In [ ]:
df_temp.info()

In [ ]:
del df_temp

Implementation part is done While very fast, it has drawbacks. For example, what happens when a factor level has a single value? Theoretically, the log-odds should be infinite in the appropriate direction i.e. p/(1 - p) tends to infinity if p = 1. And numerically, it is usually capped at a large (and inaccurate) value.

For example in above implementation for column construction_type their are 2 categories 1 has around 148637 values and other has 33 values so for category mh which has 33 values p becomes 1 and it raises the error. Then This lead us to move to next technique known as target encoding which is simpler that likelihood encoding.



<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 3.1.2 Target Encoding</h3><a id = "3.1.2" ></a>


It is same as likelihood encoding but the difference is we use average of output variable for that particular category to encode values inplace of some linear model. Lets move to the implementation.

In [ ]:
def target_encoding(df, cat_cols, target_variable = "Status"):

    for col in cat_cols:
        weight = 7
        feat = df.groupby(col)[target_variable].agg(["mean", "count"])
        mean = feat['mean']
        count = feat['count']

        smooth = (count * mean + weight * mean) / (weight + count)

        df.loc[:, col] = df.loc[:, col].map(smooth)

    return df

In [ ]:
df_temp = df_new.copy()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


## Again we have to temporarily encode variables
lb = LabelEncoder()

cat_cols = [col for col in df_temp.columns if df_temp[col].dtype == 'object']

for col in cat_cols:
    df_temp[col] = lb.fit_transform(df_temp[col])


In [ ]:
df_temp = target_encoding(df_temp, cat_cols)

In [ ]:
df_temp.head()

- Target Encoding could be good choice for binary classification but for regression it is not, because it ignores intra-category variation of the target variable. This is addressed in Bayesian Target Encoding.

- Target encoding has a tendency to overfit due to the target leakage.

- Another problem is that some of the categories have few training examples, and the mean target value for these categories may assume extreme values, so encoding these values with mean may reduce the model performance.

These issues are addressed in Bayesian Target Encoding. Which you can read from this informational blog [Target Encoding and Bayesian Target Encoding](https://towardsdatascience.com/target-encoding-and-bayesian-target-encoding-5c6a6c58ae8c)


In [ ]:
df['age'].value_counts()

<h3  style="text-align: center" class="list-group-item list-group-item-warning"> 3.1.3 Deep Learning Methods</h3><a id = "3.1.3" ></a>


Another supervised approach comes from the deep learning literature on the analysis of textual data. In this case, large amounts of text can be cut up into individual words. Rather than making each of these words into its own indicator variable, word embedding or entity embedding approaches have been developed. Similar to the dimension reduction methods, the idea is to estimate a smaller set of numeric features that can be used to adequately represent the categorical predictors.

In addition to the dimension reduction, there is the possibility that these methods can estimate semantic relationships between words so that words with similar themes (e.g., “dog”, “pet”, etc.) have similar values in the new encodings. This technique is not limited to text data and can be used to encode any type of qualitative variable.

The idea is well very simple do not extract features manually use neural network to do the hard part and just wait for the results.

<h2  style="text-align: center" class="list-group-item list-group-item-success"> 3.2 Approaches for Novel Categories</h2><a id = "3.2" ></a>



What if some new category introduce to some attribute in future how will we encode that variable then? If there is a possibility of encountering a new category in the future, one strategy would be to use the "other" category to capture new values.

While this approach may not be the most effective at extracting predictive information relative to the response for this specific category, it does enable the original model to be applied to new data without completely refitting and we do need to ensure that the "other" category is present in the training/testing data.

After assigning "other" category to novel category than we can do all kinds of encodings we studied above

This is it for this notebook in upcoming notebooks will try to cover more data preprocessing techniques like Binning for reducing noise, application of PCA in feature engineering, advance feature selection techniques and many more.

<h1 style="text-align: center" class="list-group-item list-group-item-action active">References</h1><a id = "" ></a>

- [Data Preprocessing in Machine Learning](https://www.javatpoint.com/data-preprocessing-machine-learning)
- [Easy Way of Finding and Visualizing Missing Data in Python](https://medium.datadriveninvestor.com/easy-way-of-finding-and-visualizing-missing-data-in-python-bf5e3f622dc5)
- [Visualizing Missing Values in Python is Shockingly Easy](https://towardsdatascience.com/visualizing-missing-values-in-python-is-shockingly-easy-56ed5bc2e7ea)
- [Encoding categorical variables using likelihood estimation](https://datascience.stackexchange.com/questions/11024/encoding-categorical-variables-using-likelihood-estimation)
- [Target Encoding and Bayesian Target Encoding](https://towardsdatascience.com/target-encoding-and-bayesian-target-encoding-5c6a6c58ae8c)